<a href="https://colab.research.google.com/github/C22-PS008/machine-learning/blob/main/train/named-entity-recognition/named-entity-recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependency Installation

In [1]:
!pip install transformers
!pip install tokenizers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 10.1 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 8.0 MB/s 
     |████████████████████████████████| 212 kB 66.0 MB/s 
     |████████████████████████████████| 1.1 MB 65.2 MB/s 
     |████████████████████████████████| 140 kB 72.4 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |███████████████████████████████

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 7.4 MB/s 


# Log-in and load dataset

In [8]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset
wikiann_dataset = load_dataset("wikiann","id")

In [ ]:
from datasets import load_dataset
indonlu_dataset = load_dataset("indonlu","nergrit")

In [3]:
from datasets import load_dataset
conll2033_dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
wikiann_label_names=wikiann_dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
wikiann_label_names

In [ ]:
indonlu_label_names=indonlu_dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
indonlu_label_names

In [ ]:
indonlu_feature=indonlu_dataset["train"].features
indonlu_feature

In [4]:
conll2003_label_names=conll2033_dataset["train"].features[f"ner_tags"].feature.names

In [5]:
conll2003_label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# Defining Model and Tokenizer

In [8]:
base_model="bert-base-cased"

In [ ]:
base_model="bert-base-multilingual-cased"

In [ ]:
base_model="indobenchmark/indobert-base-p1"

In [6]:
base_model="distilbert-base-cased"

In [7]:
from transformers import AutoTokenizer 

tokenizer = AutoTokenizer.from_pretrained(base_model)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

# Tokenize Dataset 





In [9]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 3325, 1154, 1734, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
def aligning_labels_with_token(labels, word_id):
  new_labels=[]
  current_word=None
  label=0
  for word in word_id:
    if word != current_word:
      current_word=word
      if word is None:
        label=-100
      else:
        label=labels[word]
      new_labels.append(label)
    elif word is None:
      new_labels.append(-100)
    else:
      label=labels[word]
      if label %2==1:
        label+=1
      new_labels.append(label)
  return new_labels

In [13]:
inputs=tokenizer(conll2033_dataset["train"][0]["tokens"], is_split_into_words=True)
labels=conll2033_dataset["train"][0]["ner_tags"]
word_ids=inputs["input_ids"]
word_id_test=inputs.word_ids(0)


In [11]:
def tokenizing_and_labeling(examples):
  tokenized_input=tokenizer(examples["tokens"],truncation=True, is_split_into_words=True)
  all_label=examples["ner_tags"]
  new_labels=[]
  for i,labels in enumerate(all_label):
    new_labels.append(aligning_labels_with_token(labels,tokenized_input.word_ids(i)))
  tokenized_input["labels"]=new_labels
  return tokenized_input

In [ ]:
tokenized_indonlu_dataset=indonlu_dataset.map(tokenizing_and_labeling,batched=True,remove_columns=indonlu_dataset["train"].column_names,)

In [12]:
tokenized_conll2003_dataset=conll2033_dataset.map(tokenizing_and_labeling,batched=True,remove_columns=conll2033_dataset["train"].column_names,)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
tokenized_dataset=tokenized_conll2003_dataset

In [ ]:
for i in range(len(tokenized_dataset["train"])):
  for j in range( len(tokenized_dataset["train"][i]['labels'] )):
    if isinstance(["train"][i]['labels'][j], str)==True:
      print(f"found on %d %d",i,j)

In [ ]:
for i in range(len(tokenized_dataset["validation"])):
  for j in range( len(tokenized_dataset"validation"][i]['labels'] )):
    if isinstance(tokenized_dataset["validation"][i]['labels'][j], str)==True:
      print(f"found on %d %d",i,j)

In [ ]:
for i in range(len(tokenized_dataset["test"])):
  for j in range( len(tokenized_dataset["test"][i]['labels'] )):
    if isinstance(tokenized_dataset["test"][i]['labels'][j], str)==True:
      print(f"found on %d %d",i,j)

# Pytorch Dataset Preprocessing and loading into the model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from torch.utils.data import DataLoader

torch_train_dataset = DataLoader(tokenize_indonlu_dataset["train"],shuffle=True,collate_fn=pytorch_data_collator,
    batch_size=16,
)
torch_validation_dataset = DataLoader(
    tokenize_indonlu_dataset["validation"], collate_fn=pytorch_data_collator, batch_size=16
)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-squad", tokenizer=tokenizer)

# We're going to do validation afterwards, so no validation mid-training
model.fit(tf_train_dataset, callbacks=[callback], epochs=num_train_epochs)

# Pytorch with Accelerator intialize

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, torch_train_dataset, torch_validation_dataset = accelerator.prepare(
    model, optimizer, torch_train_dataset, torch_validation_dataset
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(torch_train_dataset)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in torch_train_dataset:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in torch_validation_dataset:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

# Prepare dataset for Tensorflow

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer,return_tensors="tf")

In [15]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
  columns=['attention_mask','input_ids', 'labels','tokens'],
  shuffle=True,
  batch_size=16,
  collate_fn=data_collator,
)
tf_validation_dataset=tokenized_dataset["validation"].to_tf_dataset(
  columns=['attention_mask','input_ids', 'labels','tokens'],
  shuffle=False,
  batch_size=16,
  collate_fn=data_collator,
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


# Training Using Tensorflow

In [16]:
label_names=conll2003_label_names

In [17]:
id2label={str(i): label for i, label in enumerate(label_names)}
label2id={v: k for k,v in id2label.items()}

In [19]:
from transformers import TFAutoModelForTokenClassification
model=TFAutoModelForTokenClassification.from_pretrained(
    base_model,
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/338M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForTokenClassification: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import create_optimizer
import tensorflow as tf

#tf.keras.mixed_precision.set_global_policy("mixed_float16")
num_train_epochs=5
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
#model.bert.load_state_dict(model.bert.state_dict())
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16

#tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [21]:
from transformers.keras_callbacks import PushToHubCallback
callback=PushToHubCallback(output_dir="distillbert-finetuned-ner", tokenizer=tokenizer)
model.fit(tf_train_dataset,validation_data=tf_validation_dataset, callbacks=[callback], epochs=num_train_epochs,)

Cloning https://huggingface.co/chanifrusydi/distillbert-finetuned-ner into local empty directory.


Download file tf_model.h5:   0%|          | 3.47k/249M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/249M [00:00<?, ?B/s]

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


877/877 [==============================] - 78s 89ms/step - loss: 0.0638 - val_loss: 0.0732
Epoch 3/5
877/877 [==============================] - 79s 90ms/step - loss: 0.0376 - val_loss: 0.0668
Epoch 4/5
877/877 [==============================] - 75s 85ms/step - loss: 0.0241 - val_loss: 0.0707
Epoch 5/5
877/877 [==============================] - 75s 85ms/step - loss: 0.0168 - val_loss: 0.0691


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.34k/249M [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/chanifrusydi/distillbert-finetuned-ner
   c11b3f4..722c244  main -> main

